In [1]:
import csv
import datetime
import numpy as np
import matplotlib as mp
import urllib

from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.palettes import Category20
from bokeh.models import RadioButtonGroup
output_notebook()

from IPython.display import HTML

HTML('''<script> code_show=true; 
        function code_toggle() {
         if (code_show){
             $('div.input').hide();
         } else {
             $('div.input').show();
         }
         code_show = !code_show
        } 
        $( document ).ready(code_toggle);
        </script>
        <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')



/usr/local/lib/python3.7/site-packages/pandas/compat/__init__.py:84: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


Loading BokehJS ...

In [2]:
confirmed_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'
deaths_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv'
recovered_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv'
confirmed_raw = urllib.request.urlopen(confirmed_url).read().decode()
deaths_raw = urllib.request.urlopen(deaths_url).read().decode()
recovered_raw = urllib.request.urlopen(recovered_url).read().decode()

In [3]:
def arr2int(arr):
    return np.array([int(x) for x in arr])

def raw2dict(raw):
    data = ''.join([d for d in raw])
    data = data.split('\n')
    heads = []
    dict_data = {}
    
    cr = csv.reader(data, delimiter=',', quotechar='"', skipinitialspace=True)
    for row in cr:
        country = row[1]
#         print(row)
        if country == 'Country/Region':
            heads = row
        elif country in dict_data:
            dict_data[country] = dict_data[country] + arr2int(row[4::])
        else:
            dict_data[country] = arr2int(row[4::])
    
    total = np.zeros(list(dict_data.values())[0].shape)
    for d in dict_data.values():
        total = total + d
    dict_data["Total"] = total
        
    return [heads, dict_data]
[heads, confirmed_data] = raw2dict(confirmed_raw)

In [4]:
# output_file("line.html")

countries = ["Poland", "Italy", "US", "Germany", "France", "Korea, South", "Iran", "Japan", "India","Netherlands", "Canada", "Australia", "Brazil"]
# country = "Poland"

x_dates = []
for date in heads[4::]:
    d = [int(x) for x in date.split('/')]
    x_dates.append(datetime.datetime(d[2], d[0], d[1]))
# print(x_dates)
    
p = figure(
   tools="pan,box_zoom,reset,save",
   title="Confirmed cases",
    x_axis_type="datetime",
   x_axis_label='day', y_axis_label='confirmed cases'
)

# print(len(x_dates), len(confirmed_data["Poland"]))

colors = Category20[len(countries)]
for idx, country in enumerate(countries):
    l = len(confirmed_data[country])
    p.line(x_dates, confirmed_data[country], line_width=2, line_color=colors[idx], legend_label=country);
    p.legend.location = "top_left"


In [5]:
# show(radio_buttons_case_types)
show(p)

In [6]:
d = {}
for i in range(10):
    d['x' + str(i)]=list(range(i))

In [7]:
def getDataSinceNthCase(dataset, n):
    ndata = {}
    for c in dataset.keys():
        first = [i for i,e in enumerate(dataset[c]) if e >= n]
        if len(first) == 0:
            ndata[c] = []
        else:
            ndata[c] = dataset[c][first[0]::]
    return ndata
            

In [8]:
countries = confirmed_data.keys();
countries = ["Total", "Poland"] + sorted([c for c in countries if c not in {"Total", "Poland"}])

In [9]:
def getIndices(array, entries):
    idxs = []
    for i,e in enumerate(array):
        if e in entries:
            idxs.append(i)
    return idxs
getIndices(confirmed_data.keys(), ["Poland", "France", "Italy", "Japan"])

[1, 14, 66, 96]